In [1]:
import numpy as np
import pandas as pd
import gzip
import json

from pprint import pprint
from google.colab import drive
drive.mount('/content/drive/')

Mounted at /content/drive/


In [2]:
train_classification_df = pd.read_csv("drive/MyDrive/CS412_project/train-classification.csv",)
train_classification_df = train_classification_df.rename(columns={'Unnamed: 0': 'user_id', 'label': 'category'})

# Unifying labels
train_classification_df["category"] = train_classification_df["category"].apply(str.lower)
username2_category = train_classification_df.set_index("user_id").to_dict()["category"]

In [3]:
# stats about the labels
train_classification_df.groupby("category").count()

,user_id
category,
art,191
entertainment,323
fashion,299
food,511
gaming,13
health and lifestyle,503
mom and children,149
sports,113
tech,346


In [4]:
username2_category["kod8net"]

'tech'

In [5]:
train_data_path = "drive/MyDrive/CS412_project/training-dataset.jsonl"

username2posts_train = dict()
username2profile_train = dict()

username2posts_test = dict()
username2profile_test = dict()


with open(train_data_path, "rt") as fh:
  for line in fh:
    sample = json.loads(line)

    profile = sample["profile"]
    username = profile["username"]
    if username in username2_category:
      # train data info
      username2posts_train[username] = sample["posts"]
      username2profile_train[username] = profile


    else:
      # it is test data info
      username2posts_test[username] = sample["posts"]
      username2profile_test[username] = profile


In [6]:
# Profile Dataframe
train_profile_df = pd.DataFrame(username2profile_train).T.reset_index(drop=True)
test_profile_df = pd.DataFrame(username2profile_test).T.reset_index(drop=True)

pd.set_option('display.max_columns', None)
train_profile_df.head(2)

,username,id,full_name,biography,category_name,post_count,follower_count,following_count,is_business_account,is_private,is_verified,highlight_reel_count,bio_links,entities,ai_agent_type,fb_profile_biolink,restricted_by_viewer,country_block,eimu_id,external_url,fbid,has_clips,hide_like_and_view_counts,is_professional_account,is_supervision_enabled,is_guardian_of_viewer,is_supervised_by_viewer,is_supervised_user,is_embeds_disabled,is_joined_recently,business_address_json,business_contact_method,business_email,business_phone_number,business_category_name,overall_category_name,category_enum,is_verified_by_mv4b,is_regulated_c18,profile_pic_url,should_show_category,should_show_public_contacts,show_account_transparency_details,profile_picture_base64
0,deparmedya,3170700063,Depar Medya,#mediaplanning #mediabuying #sosyalmedya,Local business,None,1167,192,True,False,False,6,"""[{'title': '', 'lynx_url': 'https://l.instagr...",#mediaplanning #mediabuying #sosyalmedya,None,None,None,False,110133017051179,http://www.deparmedya.com/,17841403241896337,True,False,True,False,False,False,False,False,False,"""{\""city_name\"": null, \""city_id\"": null, \""la...",CALL,None,None,None,None,LOCAL,False,False,https://instagram.fsaw2-3.fna.fbcdn.net/v/t51....,True,True,True,/9j/4AAQSkZJRgABAQAAAQABAAD/2wBDAAgGBgcGBQgHBw...
1,kafesfirin,266439571,KAFES FIRIN,📍Söğütözü📍FTZ AVM\n🛒Ankara macro▲center v...,Brand,None,11997,17,True,False,False,15,"""[{'title': '', 'lynx_url': 'https://l.instagr...",📍Söğütözü📍FTZ AVM\n🛒Ankara macro▲center v...,None,None,None,False,111783696881731,http://www.kafesfirin.com/,17841400227134163,True,False,True,False,False,False,False,False,False,"""{\""city_name\"": null, \""city_id\"": null, \""la...",CALL,None,None,None,None,BRAND,False,False,https://instagram.fada1-13.fna.fbcdn.net/v/t51...,True,True,True,/9j/4AAQSkZJRgABAQAAAQABAAD/2wBDAAgGBgcGBQgHBw...


In [7]:
pd.set_option('display.max_columns', 10)
test_profile_df.head(2)

,username,id,full_name,biography,category_name,...,profile_pic_url,should_show_category,should_show_public_contacts,show_account_transparency_details,profile_picture_base64
0,beyazyakaliyiz,8634457436,Selam Beyaz Yakalı,Beyaz yakalıların dünyasına hoşgeldiniz 😀😀😀,Personal blog,...,https://instagram.fist6-1.fna.fbcdn.net/v/t51....,True,True,True,/9j/4AAQSkZJRgABAQAAAQABAAD/2wBDAAgGBgcGBQgHBw...
1,totalenergies_istasyonlari,7066643793,TotalEnergies İstasyonları,TotalEnergies İstasyonları resmi Instagram hes...,Energy Company,...,https://instagram.fsaw2-1.fna.fbcdn.net/v/t51....,True,True,True,/9j/4AAQSkZJRgABAQAAAQABAAD/2wBDAAgGBgcGBQgHBw...


In [8]:
!pip install emoji
!pip install stanza

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 586.9/586.9 kB 11.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.1/1.1 MB 20.6 MB/s eta 0:00:00


In [81]:
import nltk
from nltk.corpus import stopwords
import re
import emoji
import stanza

stanza.download('tr')
nlp = stanza.Pipeline('tr', processors='tokenize,lemma', use_gpu=True, tokenize_pretokenized=True, lemma_dict_only=True, batch_size=50)

nltk.download('stopwords')
turkish_stopwords = stopwords.words('turkish')

INFO:stanza:Downloaded file to /root/stanza_resources/resources.json
INFO:stanza:Downloading default packages for language: tr (Turkish) ...
INFO:stanza:File exists: /root/stanza_resources/tr/default.zip
INFO:stanza:Finished downloading models and saved to /root/stanza_resources
INFO:stanza:Checking for updates to resources.json in case models have been updated.  Note: this behavior can be turned off with download_method=None or download_method=DownloadMethod.REUSE_RESOURCES


INFO:stanza:Downloaded file to /root/stanza_resources/resources.json
INFO:stanza:Loading these models for language: tr (Turkish):
| Processor | Package       |
-----------------------------
| tokenize  | imst          |
| lemma     | imst_nocharlm |

INFO:stanza:Using device: cuda
INFO:stanza:Loading: tokenize
INFO:stanza:Loading: lemma
INFO:stanza:Done loading processors!
[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [77]:
emoji_pattern = re.compile(
    r'['
    r'\U0001F600-\U0001F64F'  # Emoticons
    r'\U0001F300-\U0001F5FF'  # Miscellaneous Symbols and Pictographs
    r'\U0001F680-\U0001F6FF'  # Transport and Map Symbols
    r'\U0001F1E0-\U0001F1FF'  # Flags
    r'\U00002702-\U000027B0'  # Dingbats
    r'\U000024C2-\U0001F251'  # Enclosed characters
    r'\U0001F900-\U0001F9FF'  # Supplemental Symbols and Pictographs
    r'\U0001FA70-\U0001FAFF'  # Symbols and Pictographs Extended-A
    r'\U00002600-\U000026FF'  # Miscellaneous Symbols
    r'\U00002B50-\U00002B55'  # Stars and circles
    r']',
    re.UNICODE
)

alphabetic_pattern = r'[a-zçğıöşü\s]'

non_emoji_alphabetic_pattern = re.compile(
    r'[^'
    r'\U0001F600-\U0001F64F'
    r'\U0001F300-\U0001F5FF'
    r'\U0001F680-\U0001F6FF'
    r'\U0001F1E0-\U0001F1FF'
    r'\U00002702-\U000027B0'
    r'\U000024C2-\U0001F251'
    r'\U0001F900-\U0001F9FF'
    r'\U0001FA70-\U0001FAFF'
    r'\U00002600-\U000026FF'
    r'\U00002B50-\U00002B55'
    r'a-zçğıöşü\s'
    r']',
    re.UNICODE
)

In [86]:
from tqdm import tqdm

def clean_text(text: str):
  text = text.casefold()
  text = re.sub(r'http\S+|www\S+|https\S+', ' ', text)
  text = re.sub(non_emoji_alphabetic_pattern, ' ', text)
  text = emoji_pattern.sub(r' \g<0> ', text)
  text = re.sub(r'\s+', ' ', text).strip()
  return text

def preprocess_text(text: str):
  text = clean_text(text)

  doc = nlp(text)

  lemmas = []
  for sentence in doc.sentences:
    for word in sentence.words:
      if re.match(emoji_pattern, word.text):
        lemmas.append(word.text)
      elif word.lemma is not None:
        lemmas.append(word.lemma)

  text = ' '.join(lemmas)

  return text

corpus = []
train_usernames = []

for username, posts in tqdm(username2posts_train.items(), desc="Processing Users"):
  train_usernames.append(username)

  # aggregating the posts per user
  cleaned_captions = []
  for post in posts:
    post_caption = post.get("caption", "")
    if post_caption is None:
      continue

    post_caption = preprocess_text(post_caption)

    if post_caption:
      cleaned_captions.append(post_caption)

  profile = username2profile_train[username]

  bio = profile.get("biography", "")
  if bio:
    cleaned_captions.append(preprocess_text(bio))

  # joining the posts of each user with a \n
  user_post_captions = "\n".join(cleaned_captions)
  corpus.append(user_post_captions)

Processing Users: 100%|██████████| 2741/2741 [01:44<00:00, 26.34it/s]


In [87]:
print(corpus[0])
print(len(corpus))

cumhuriyetimizin yıl kutlu ol ♾ ️ 🇹 🇷
oriflame duologi lansmanı isveçtengelengüzellik oriflameilesaçbakımdevrimi
oriflameilesaçbakımdevrimi ✌ ️
✌ ️ oriflameilesaçbakımdevrimi agustos oriflameturkiye
agustos oriflameturkiye duoloji
oriflame duoloji
muhteşem saç sır duoloji oriflameturkiye
goe elektirikli motorsiklet ile sürdürülebilir bir yaşam elçisi olun goeilegeleceğeyönver goeilesürdürülebilirgelecek easytorideeasytolove goe mobility 🏍 ️
haziran yazamerhaba feridaistanbul
haziran yazamerhaba
feridaistanbul yazamerhaba ✌ ️ 🌸 feridacadde
jumbopatisserie nin edible art konseptiyle özel olarak hazırla tatlıları ve baci milano nun şık ve renkli dünya kahvaltıdayız jumboturkiye
easy to love easy to ride ❤ ️ goe mobility
figamortr
mustafa kemal atatürk ü sevgi saygı ve özlem anıyoruz
notecosmetiqueturkiye event kendimenote
notecosmetiqueturkiye
otoshops outdoor çalış otoshopsturkiye
mad parfüm outdoor çalışmalarımız madparfumeurofficial açıkhavareklam letsgetmad
notecosmetiqueturkiye kendi

In [88]:
from sklearn.feature_extraction.text import CountVectorizer, TfidfVectorizer

vectorizer = TfidfVectorizer(
    stop_words=turkish_stopwords,
    max_features=5000,
    ngram_range=(1, 3),
    tokenizer=lambda text: re.findall(r'\w+|[^\s\w]', text)
    )

# fit the vectorizer
vectorizer.fit(corpus)


# transform the data into vectors
x_post_train = vectorizer.transform(corpus)
y_train_categories = [username2_category.get(uname, "NA") for uname in train_usernames]

In [89]:
# Making sure everything is fine
assert y_train_categories.count("NA") == 0

In [90]:
feature_names = vectorizer.get_feature_names_out()
feature_names

array(['2', 'abdullah', 'abone', ..., '\U0001faf6', '\U0001faf6 🏻',
       '\U0001faf6 🏼'], dtype=object)

In [91]:
df_tfidf = pd.DataFrame(x_post_train.toarray(), columns=feature_names)

pd.set_option('display.max_columns', None)
df_tfidf.head(2)

2  abdullah  abone  about   ac  acar  acele  acil  acil şifalar  acı  \
0  0.0       0.0    0.0    0.0  0.0   0.0    0.0   0.0           0.0  0.0   
1  0.0       0.0    0.0    0.0  0.0   0.0    0.0   0.0           0.0  0.0   

         ad  ada  adam  adana  adapazarı  adası  aday  add  adet  \
0  0.032314  0.0   0.0    0.0        0.0    0.0   0.0  0.0   0.0   
1  0.000000  0.0   0.0    0.0        0.0    0.0   0.0  0.0   0.0   

   adet yumurta  adeta  adidas      adlı  adres  adresinden  adresini  \
0           0.0    0.0     0.0  0.050701    0.0         0.0       0.0   
1           0.0    0.0     0.0  0.000000    0.0         0.0       0.0   

   adresini ziyaret  adventure  adım  adım adım  adım at  adımlar  aegean  \
0               0.0        0.0   0.0        0.0      0.0      0.0     0.0   
1               0.0        0.0   0.0        0.0      0.0      0.0     0.0   

   aesthetic  aesthetics  afad  afet  afiyet  afiyet ol  afiyetle  afrika  \
0        0.0         0.0   0.0   0.0     0.0        0.0       0.0     0.0   
1        0.0         0.0   0.0   0.0     0.0        0.0       0.0     0.0   

   after  afyon   ag  again  agency  ahmet  ahşap   ai  aile  aile olarak  \
0    0.0    0.0  0.0    0.0     0.0    0.0    0.0  0.0   0.0          0.0   
1    0.0    0.0  0.0    0.0     0.0    0.0    0.0  0.0   0.0          0.0   

   air  ait  ajans   ak  ak parti  akademi  akademik  akdeniz  aksesuar  \
0  0.0  0.0    0.0  0.0       0.0      0.0       0.0      0.0       0.0   
1  0.0  0.0    0.0  0.0       0.0      0.0       0.0      0.0       0.0   

   aktif  aktivite  akyaka  akü  akıl    akıllı  akşam  akşam yemek  \
0    0.0       0.0     0.0  0.0   0.0  0.099743    0.0          0.0   
1    0.0       0.0     0.0  0.0   0.0  0.000000    0.0          0.0   

   akşamyemeği  akşamı        al  al bir  alabilirsiniz  alan  alan uzman  \
0          0.0     0.0  0.018571     0.0            0.0   0.0         0.0   
1          0.0     0.0  0.000000     0.0            0.0   0.0         0.0   

   alanya  alaçatı  ali  alkol  all  all our  all the  allah  allah tan  \
0     0.0      0.0  0.0    0.0  0.0      0.0      0.0    0.0        0.0   
1     0.0      0.0  0.0    0.0  0.0      0.0      0.0    0.0        0.0   

   allah tan rahmet  almanya  alsancak  also  alt  alt al  alternatif  altı  \
0               0.0      0.0       0.0   0.0  0.0     0.0         0.0   0.0   
1               0.0      0.0       0.0   0.0  0.0     0.0         0.0   0.0   

   altın  always  alıcı  alın  alıs  alıyoruz  alışveriş   am  amazing  \
0    0.0     0.0    0.0   0.0   0.0       0.0        0.0  0.0      0.0   
1    0.0     0.0    0.0   0.0   0.0       0.0        0.0  0.0      0.0   

   amazingdestination  amazingdestination familyhotel  amazingplaces  \
0                 0.0                             0.0            0.0   
1                 0.0                             0.0            0.0   

   amazingplaces keepcalmandtravel  \
0                              0.0   
1                              0.0   

   amazingplaces keepcalmandtravel amazingdestination      amaç  amaçlı  \
0                                                0.0   0.040423     0.0   
1                                                0.0   0.000000     0.0   

   amaçlıdır  ambalaj  ameliyat  amerika  amsterdam        an  \
0        0.0      0.0       0.0      0.0        0.0  0.000000   
1        0.0      0.0       0.0      0.0        0.0  0.036135   

   an unforgettable  ana  anadolu  anahtar  analiz  analizi  anatolium  ancak  \
0               0.0  0.0      0.0      0.0     0.0      0.0        0.0    0.0   
1               0.0  0.0      0.0      0.0     0.0      0.0        0.0    0.0   

   and  and the  and we  animal    ankara  ankara istanbul  anla     anlam  \
0  0.0      0.0     0.0     0.0  0.000000              0.0   0.0  0.042673   
1  0.0      0.0     0.0     0.0  0.056894              0.0   0.0  0.000000   

   anlam gel  anlamlı  anları  anlat  anlayış  anma  anma ge

In [92]:
pd.set_option('display.max_columns', 10)
df_tfidf.shape

(2741, 5000)

In [103]:
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import MinMaxScaler

# scaler = MinMaxScaler()

x_train, x_val, y_train, y_val = train_test_split(df_tfidf, y_train_categories, test_size=0.2, stratify=y_train_categories)

# scaler.fit(x_train)

# x_train = scaler.transform(x_train)
# x_val = scaler.transform(x_val)

In [94]:
x_train.shape

(2192, 5000)

In [95]:
x_val.shape

(549, 5000)

In [104]:
from sklearn.naive_bayes import MultinomialNB, ComplementNB, BernoulliNB, GaussianNB, CategoricalNB
from sklearn.ensemble import RandomForestClassifier
from sklearn import svm
from sklearn.model_selection import GridSearchCV
from sklearn.metrics import accuracy_score, classification_report

model = svm.LinearSVC(loss="hinge", class_weight='balanced')
model.fit(x_train, y_train)

LinearSVC(class_weight='balanced', loss='hinge')

In [105]:
y_train_pred = model.predict(x_train)

print("Accuracy:", accuracy_score(y_train, y_train_pred))
print("\nClassification Report:")
print(classification_report(y_train, y_train_pred, zero_division=0))

Accuracy: 0.9548357664233577

Classification Report:
                      precision    recall  f1-score   support

                 art       0.96      0.92      0.94       153
       entertainment       0.97      0.91      0.94       258
             fashion       0.91      0.97      0.94       239
                food       0.98      0.98      0.98       409
              gaming       0.91      1.00      0.95        10
health and lifestyle       0.98      0.92      0.95       402
    mom and children       0.90      0.98      0.94       119
              sports       0.96      1.00      0.98        90
                tech       0.93      0.97      0.95       277
              travel       0.95      0.97      0.96       235

            accuracy                           0.95      2192
           macro avg       0.95      0.96      0.95      2192
        weighted avg       0.96      0.95      0.95      2192



In [106]:
y_val_pred = model.predict(x_val)

print("Accuracy:", accuracy_score(y_val, y_val_pred))
print("\nClassification Report:")
print(classification_report(y_val, y_val_pred, zero_division=0))

Accuracy: 0.6593806921675774

Classification Report:
                      precision    recall  f1-score   support

                 art       0.39      0.32      0.35        38
       entertainment       0.49      0.31      0.38        65
             fashion       0.62      0.75      0.68        60
                food       0.87      0.88      0.88       102
              gaming       0.00      0.00      0.00         3
health and lifestyle       0.72      0.58      0.64       100
    mom and children       0.51      0.67      0.58        30
              sports       0.50      0.78      0.61        23
                tech       0.70      0.81      0.75        69
              travel       0.66      0.73      0.69        59

            accuracy                           0.66       549
           macro avg       0.55      0.58      0.56       549
        weighted avg       0.65      0.66      0.65       549



In [108]:
test_usernames = []
test_corpus = []
test_business_categories = []
test_category_enums = []

for username, posts in tqdm(username2posts_test.items(), desc="Processing Users"):
  test_usernames.append(username)
  # aggregating the posts per user
  cleaned_captions = []
  for post in posts:
    post_caption = post.get("caption", "")
    if post_caption is None:
      continue

    post_caption = preprocess_text(post_caption)

    if post_caption != "":
      cleaned_captions.append(post_caption)

    profile = username2profile_train.get(username, dict())

    bio = profile.get("biography", "")
    if (bio is not None) and (bio != ""):
      cleaned_captions.append(preprocess_text(bio))

    business_category = profile.get("business_category_name", "None")
    if business_category is None:
      business_category = "None"
    test_business_categories.append(business_category)

    category_enum = profile.get("category_enum", "NONE")
    if category_enum is None:
      category_enum = "NONE"
    test_category_enums.append(category_enum)

  user_post_captions = "\n".join(cleaned_captions)
  test_corpus.append(user_post_captions)


# Just transforming! No Fitting!!!!!
x_post_test = vectorizer.transform(test_corpus)

Processing Users: 100%|██████████| 2674/2674 [01:51<00:00, 24.01it/s]


In [109]:
# let's take a look at the first 5 lines of the file
test_data_path = "drive/MyDrive/CS412_project/test-classification-round3.dat"
!head -n 5 "$test_data_path"

print("*****")

test_unames = []
with open(test_data_path, "rt") as fh:
  for line in fh:
    test_unames.append(line.strip())

print(test_unames[:5])

livapastanesi
barisgross
tusasshop
etolyadigital
tugrulonur
*****
['livapastanesi', 'barisgross', 'tusasshop', 'etolyadigital', 'tugrulonur']


In [110]:
x_test = []

for uname in test_unames:
  try:
    index = test_usernames.index(uname)
    x_test.append(x_post_test[index].toarray()[0])
  except Exception as e:
    try:
      index = train_usernames.index(uname)
      x_test.append(x_post_train[index].toarray()[0])
    except Exception as e:
      print(uname)

In [111]:
test_pred = model.predict(x_test)

output = dict()
for index, uname in enumerate(test_unames):
  output[uname] = test_pred[index]

/usr/local/lib/python3.10/dist-packages/sklearn/utils/validation.py:2739: UserWarning: X does not have valid feature names, but LinearSVC was fitted with feature names
  warnings.warn(


In [ ]:
with open("drive/MyDrive/CS412_project/prediction-classification-round3.json", "w") as of:
  json.dump(output, of, indent=4)

In [ ]:
from datetime import datetime
def time_in_seconds(timestamp):
  dt = datetime.strptime(timestamp, "%Y-%m-%d %H:%M:%S")
  return dt.hour * 3600 + dt.minute * 60 + dt.second

In [ ]:
from sklearn.linear_model import LinearRegression
from sklearn import svm

username2regressors = dict()

def predict_like_count(username, current_post):

  global username2regressors

  if username in username2regressors:
    regressor = username2regressors[username]

    comment_count = current_post.get("comments_count", 0)

    time = time_in_seconds(current_post["timestamp"])
    day  = datetime.strptime(current_post["timestamp"], "%Y-%m-%d %H:%M:%S").weekday()

    sin_time = np.sin(2 * np.pi * time / 86400)
    cos_time = np.cos(2 * np.pi * time / 86400)
    sin_day = np.sin(2 * np.pi * day / 7)
    cos_day = np.cos(2 * np.pi * day / 7)

    result = regressor.predict([[comment_count, sin_time, cos_time, sin_day, cos_day]])[0]
    return max(result, 0)

  # Otherwise...

  posts = []
  if username in username2posts_train:
    posts = username2posts_train[username]
  elif username in username2posts_test:
    posts = username2posts_test[username]
  else:
    print(f"No data available for {username}")
    return 0

  if not posts:
    print(f"No data available for {username}")
    return 0

  comment_counts = [post.get("comments_count", 0) if post.get("comments_count", 0) is not None else 0 for post in posts]
  like_counts    = [post.get("like_count", 0) if post.get("like_count", 0) is not None else 0 for post in posts]

  times = np.array([time_in_seconds(post["timestamp"]) if post["timestamp"] is not None else 0 for post in posts])
  days  = np.array([datetime.strptime(post["timestamp"], "%Y-%m-%d %H:%M:%S").weekday() if post["timestamp"] is not None else 0 for post in posts])



  sin_times = np.sin(2 * np.pi * times / 86400)
  cos_times = np.cos(2 * np.pi * times / 86400)
  sin_days = np.sin(2 * np.pi * times / 7)
  cos_days = np.cos(2 * np.pi * times / 7)

  x_train = list(zip(comment_counts, sin_times, cos_times, sin_days, cos_days))
  y_train = like_counts

  regressor = svm.SVR()
  regressor.fit(x_train, y_train)

  username2regressors[username] = regressor

  comment_count = current_post.get("comments_count", 0)

  time = time_in_seconds(current_post["timestamp"])
  day  = datetime.strptime(current_post["timestamp"], "%Y-%m-%d %H:%M:%S").weekday()

  sin_time = np.sin(2 * np.pi * time / 86400)
  cos_time = np.cos(2 * np.pi * time / 86400)
  sin_day = np.sin(2 * np.pi * day / 7)
  cos_day = np.cos(2 * np.pi * day / 7)

  result = regressor.predict([[comment_count, sin_time, cos_time, sin_day, cos_day]])[0]
  return max(result, 0)

In [ ]:
def log_mse_like_counts(y_true, y_pred):
  """
  Calculate the Log Mean Squared Error (Log MSE) for like counts (log(like_count + 1)).

  Parameters:
  - y_true: array-like, actual like counts
  - y_pred: array-like, predicted like counts

  Returns:
  - log_mse: float, Log Mean Squared Error
  """
  # Ensure inputs are numpy arrays
  y_true = np.array(y_true)
  y_pred = np.array(y_pred)

  # Log transformation: log(like_count + 1)
  log_y_true = np.log1p(y_true)
  log_y_pred = np.log1p(y_pred)

  # Compute squared errors
  squared_errors = (log_y_true - log_y_pred) ** 2

  # Return the mean of squared errors
  return np.mean(squared_errors)

In [ ]:
y_like_count_train_true = []
y_like_count_train_pred = []

for uname, posts in username2posts_train.items():
  for post in posts:
    pred_val = predict_like_count(uname, post)
    true_val = post.get("like_count", 0)
    if true_val is None:
      true_val = 0

    y_like_count_train_true.append(true_val)
    y_like_count_train_pred.append(pred_val)

print(f"Log MSE Train = {log_mse_like_counts(y_like_count_train_true, y_like_count_train_pred)}")

Log MSE Train = 0.7946997339108318


In [ ]:
from sklearn.metrics import mean_squared_error
mse_train = mean_squared_error(y_like_count_train_true, y_like_count_train_pred)
print(f"MSE Train = {mse_train}")

MSE Train = 770623802.9586772


In [ ]:
y_like_count_test_true = []
y_like_count_test_pred = []

for uname, posts in username2posts_test.items():
  for post in posts:
    pred_val = predict_like_count(uname, post)
    true_val = post.get("like_count", 0)
    if true_val is None:
      true_val = 0

    y_like_count_test_true.append(true_val)
    y_like_count_test_pred.append(pred_val)

print(f"Log MSE Test = {log_mse_like_counts(y_like_count_test_true, y_like_count_test_pred)}")

Log MSE Test = 0.5598439498889954


In [ ]:
path = "drive/MyDrive/CS412_project/test-regression-round3.jsonl"
output_path = "drive/MyDrive/CS412_project/prediction-regression-round3.json"

to_predict_like_counts_usernames = []
output = dict()
with open(path, "rt") as fh:
  for line in fh:
    sample = json.loads(line)

    # let's predict
    pred_val = predict_like_count(sample["username"], sample)

    output[sample["id"]] = round(pred_val)

with open(output_path, "wt") as of:
  json.dump(output, of)

No data available for touchdownistanbul
